# Signal to Noise Ratio (SNR) estimation for 3D MRIs with Rician noise

This scripts generates 3D magnetic resonance images (MRIs) corrupted with Rician noise from a set of noiseless MRIs. Then the noisy images are analyzed in order to estimate their Signal to Noise Ratio (SNR).

In [2]:
# Import the library
import numpy as np
import nibabel as nib
import os
from glob import glob 
import scipy
from scipy import stats
import pandas as pd

# Add Rician noise to a 3D MRI
# Inputs:
#     img3D=The noiseless 3D MRI as a numpy 3D array
#     StandardDeviation=The standard deviation of the Rician noise
#     NoiseScale=Scaling parameter for the noise
#     fixedSeed=Pseudorandom seed to be employed in the noise generation
# Output:
#     module=The noisy module 3D MRI as a numpy 3D array
def ricianDistribution(img3D, StandardDeviation, NoiseScale, fixedSeed):
    if StandardDeviation==0:
        return img3D
    np.random.seed(seed=fixedSeed)
    distributionRandom1 = np.random.normal(size = img3D.shape)
    distributionRandom2 = np.random.normal(size = img3D.shape)
    # Generate the noise
    x = StandardDeviation * NoiseScale * distributionRandom1 + img3D
    y = StandardDeviation * NoiseScale * distributionRandom2
    module = np.sqrt(pow(x, 2) + pow(y, 2))
    # Return the image with noise
    return  module

In [3]:
# Define the dataset to use, choosing one of the directories: 20OASIS_SynapseWeb_brain, OASIS-TRT-20_volumes, 12HLN, NKI-TRT-20_volumes, NKI-RS-22_volumes, MMRR-21_volumes
repository = '12HLN' #20OASIS_SynapseWeb_brain #OASIS-TRT-20_volumes #12HLN #NKI-TRT-20_volumes #NKI-RS-22_volumes #MMRR-21_volumes
inputDir = ('../input') 
outputDir = ('../output')

# Path with input images
dataMRI = os.path.join(inputDir, repository)
typeImage = 't1weighted.nii.gz' #'t1weighted_brain.nii.gz' #'t1weighted.nii.gz'
pathMRI = sorted(glob(dataMRI + '/**/' + typeImage, recursive=True))
SIZE = len(pathMRI)



extension = '_SNR.xlsx'
outputFile = repository + extension
dataset_path = os.path.join(outputDir, outputFile)

In [27]:
repository = 'T1_AXIAL'
outputDir = ('../output')


# name of images
input_csv = ('/home/rosammq/PROJECTS/Axioms2023/input/csv/fastMRI_brain_DICOM_T1_AXIAL_.csv')
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(input_csv, index_col=None, header=None)
# Directory where you want to search for files
directory = '/home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI'
# Auxiliar variable
img_list = []

for row_idx, row in df.iterrows():
    file_path = directory + '/' + row.iloc[0]
    print(f"MRI in the path: {file_path}")
    img_list.append(file_path)


pathMRI = img_list
SIZE = len(img_list)
print(f"Size is {SIZE}")

extension = '_SNR.xlsx'
outputFile = repository + extension
dataset_path = os.path.join(outputDir, outputFile)

MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20101127000000_5.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20100421000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20121122000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20121126000000_6.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20161016000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20170206000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20150106000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_20150122000000_4.nii
MRI in the path: /home/rosammq/DATA/brain_FastMRI_DICOM/NIFTI/fastMRI_brain_DICOM_T1_AXIAL_2016060900000

In [4]:
# Prepare the experiment parameters
NumNoiseLevels=20
NumNoiselessImages=SIZE
MaxNoiseLevel=0.4

# Generate the randomly chosen noise levels
np.random.seed(seed=1)
NoiseLevels = np.linspace(0,MaxNoiseLevel,NumNoiseLevels+1)[1:]

# Loop for all noiseless images
SNR = np.zeros((NumNoiselessImages,NumNoiseLevels))

for NdxImage in range(0,NumNoiselessImages):

  # Load the noiseless 3D MRI
  NoiselessImage = nib.load(pathMRI[NdxImage]).get_fdata()

  # The interquartile range is employed as the noise scaling parameter. This way the
  # added noise level is robust against outliers in the values of the noiseless images.
  MyIQR=scipy.stats.iqr(NoiselessImage[NoiselessImage!=0])

  # Loop for all noise levels
  for NdxNoise in range(0,NumNoiseLevels):

    # Apply Rician noise
    NoiseLevel=NoiseLevels[NdxNoise]
    NoisyImage = ricianDistribution(img3D=NoiselessImage,StandardDeviation=NoiseLevel,NoiseScale=MyIQR,fixedSeed=NdxImage*NumNoiseLevels+NdxNoise)
    Noise = ricianDistribution(img3D=np.zeros_like(NoiselessImage),StandardDeviation=NoiseLevel,NoiseScale=MyIQR,fixedSeed=NdxImage*NumNoiseLevels+NdxNoise)

    # Compute the squared amplitude of the noisy image and the noise. The amplitude is the square root of the mean over the image of the squares of the voxel values.
    SquaredAmplitudeNoisyImage = np.mean(np.power(NoisyImage,2))
    SquaredAmplitudeNoise = np.mean(np.power(Noise,2))

    # Compute the Signal to Noise Ratio
    SNR[NdxImage,NdxNoise] = SquaredAmplitudeNoisyImage / SquaredAmplitudeNoise


KeyboardInterrupt: 

In [5]:
NoiseLevels

array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 , 0.22,
       0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 ])

In [43]:
SNR

array([[544.1822584 , 136.72863595,  61.33236484,  34.9373848 ,
         22.71601178,  16.08341084,  12.08313443,   9.48176323,
          7.69952468,   6.42282604,   5.48880667,   4.76942458,
          4.21206274,   3.76789098,   3.41079771,   3.12047336,
          2.8775163 ,   2.67578613,   2.50350988,   2.35685361],
       [466.3981219 , 117.38664853,  52.74471309,  30.1065312 ,
         19.6200177 ,  13.93160934,  10.50422717,   8.28208811,
          6.74747772,   5.65694001,   4.84778317,   4.2345252 ,
          3.75508292,   3.37577794,   3.06886745,   2.81876216,
          2.61141172,   2.43715677,   2.28964888,   2.16320274],
       [425.7347149 , 107.22026312,  48.20256809,  27.5632715 ,
         17.994348  ,  12.80686772,   9.67365672,   7.63913382,
          6.24371241,   5.25244104,   4.5108906 ,   3.95163376,
          3.51593081,   3.16848129,   2.88824831,   2.65947186,
          2.47033165,   2.31156999,   2.17739059,   2.06295944],
       [535.4321924 , 134.63130171,  

In [41]:
# Compute the average SNR for all images, for each noise level
AverageSNR = np.mean(SNR,axis=0)
print(AverageSNR)

[521.80381303 131.18649163  58.85861464  33.54756122  21.83017714
  15.46735805  11.62693943   9.13770064   7.42821255   6.20730914
   5.30381062   4.61624731   4.08094051   3.65661852   3.31438724
   3.03397622   2.80181144   2.60759871   2.44261966   2.30184088]


In [7]:
df1 = pd.DataFrame(NoiseLevels)
df1.to_excel('kk.xlsx', header=False, index=True)

In [45]:
df = pd.DataFrame(AverageSNR)
df.to_excel(dataset_path, header=False, index=True+1)
df  

,0
0,521.803813
1,131.186492
2,58.858615
3,33.547561
4,21.830177
5,15.467358
6,11.626939
7,9.137701
8,7.428213
9,6.207309


kk

In [34]:
# Compute the average SNR for all images, for each noise level
AverageSNR = np.mean(SNR,axis=0)
print(AverageSNR)

[556.23779534 139.79187354  62.69166407  35.69953406  23.20786188
  16.42466101  12.33055776   9.67619939   7.85394624   6.55162323
   5.58871321   4.85562505   4.28492963   3.83280732   3.46735437
   3.16875957   2.92114085   2.71405864   2.53804636   2.388232  ]


In [30]:
# Compute the average SNR for all images, for each noise level
AverageSNR = np.mean(SNR,axis=0)
print(AverageSNR)

[747.95299223 187.64998481  83.97786447  47.67708994  30.87535005
  21.7472932   16.2411673   12.67033085  10.21997421   8.46708723
   7.17136062   6.18637254   5.41963161   4.8098224    4.31874462
   3.91764104   3.58396936   3.30569346   3.06876022   2.86726423]


In [10]:
# Compute the average SNR for all images, for each noise level
AverageSNR = np.mean(SNR,axis=0)
print(AverageSNR)

[521.80381303 131.18649163  58.85861464  33.54756122  21.83017714
  15.46735805  11.62693943   9.13770064   7.42821255   6.20730914
   5.30381062   4.61624731   4.08094051   3.65661852   3.31438724
   3.03397622   2.80181144   2.60759871   2.44261966   2.30184088]


In [35]:
df = pd.DataFrame(AverageSNR)
df.to_excel(dataset_path, header=False, index=False)
df  

,0
0,556.237795
1,139.791874
2,62.691664
3,35.699534
4,23.207862
5,16.424661
6,12.330558
7,9.676199
8,7.853946
9,6.551623
